In [1]:
import cv2
import numpy as np

# Buka gambar as grayscale (biar sederhana)
image = cv2.imread('gambar_asli.jpg', cv2.IMREAD_GRAYSCALE)
watermark = cv2.imread('watermark.png', cv2.IMREAD_GRAYSCALE)

# Resize watermark agar lebih kecil dari gambar
watermark = cv2.resize(watermark, (64, 64))  # bisa disesuaikan

In [2]:
def apply_dct(img):
    img = np.float32(img) / 255.0
    return cv2.dct(img)

def apply_idct(img):
    return cv2.idct(img)

# Ambil blok kiri atas untuk ditandai (contoh 128x128)
block = image[0:128, 0:128]
dct_block = apply_dct(block)


In [3]:
# Normalisasi watermark biar 0–1
wm = np.float32(watermark) / 255.0
alpha = 0.1  # kekuatan watermark (jangan terlalu besar)

# Sisipkan ke bagian frekuensi tengah (misal baris 20–84, kolom 20–84)
dct_block[20:84, 20:84] += alpha * wm


In [4]:
# Balikkan ke ruang gambar
idct_block = apply_idct(dct_block)
idct_block = np.uint8(idct_block * 255.0)

# Ganti blok awal dengan yang sudah diberi watermark
watermarked_image = image.copy()
watermarked_image[0:128, 0:128] = idct_block

cv2.imwrite("hasil_watermarked.jpg", watermarked_image)


True

In [ ]:
def extract_watermark(watermarked_img):
    block = watermarked_img[0:128, 0:128]
    dct_block = apply_dct(np.float32(block) / 255.0)
    extracted = (dct_block[20:84, 20:84]) / alpha
    return np.uint8(np.clip(extracted * 255.0, 0, 255))

img_with_wm = cv2.imread('hasil_watermarked.jpg', cv2.IMREAD_GRAYSCALE)
extracted_wm = extract_watermark(img_with_wm)

cv2.imwrite("extracted_watermark.jpg", extracted_wm)
